In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from pandas import Series, DataFrame
import pandas as pd

In [7]:
# driver = webdriver.Chrome('/Users/jungwoosong/Documents/python/chromedriver')
driver = webdriver.Firefox(executable_path="C://Users//SongJungwoo//Documents//GitHub//amazon_skills_scraper/geckodriver_win.exe")
# driver = webdriver.Firefox(executable_path='C:/Users/SongJungwoo/Documents/GitHub/amazon_skills_scraper/geckodriver')

In [10]:
driver.set_window_position("0","0")
driver.set_window_size("1920","1080")

# Categories

### Business & Finance

In [11]:
category = 'Business & Finance'
cur_category = category

### Total Scraping

In [12]:
cur_page = 'https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1503558721&rnid=13727922011'
category = 'Business & Finance'
cur_category = category
driver.get(cur_page)
skills = list()
reviews = list()
comments = list()
exitFlag = False
# for i in range(2):
while (exitFlag == False):
    time.sleep(2)
    cur_page = driver.current_url
    hrefs = list()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
    for i in range(0,len(results)):
#     for i in range(0,len(results)-8): #테스트용도로 페이지마다 8개씩만
        hrefs.append(results[i]['href'])
#     for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
#         hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))
#     time.sleep(2)
    for link in hrefs:
        time.sleep(1)
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
        author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
        rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
        ratings_percentage_raw = soup.select("#histogramTable > tbody")
        review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
        description_raw = soup.select("#a2s-description > span")
        commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
        invocation_raw = soup.select("#a2s-skill-details > ul > li")
        # skill id
        # timestamp
        skill = {}
        skill['category'] = cur_category
        skill['title'] = name_raw[0].get_text().strip()
        skill['author'] = author_raw[0].get_text().strip()
        if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
            print("Rating exist")
            print("rating_raw = " + rating_raw[0].get_text())
            skill['rating'] = rating_raw[0].get_text()
            ratings_percentages = list()
            for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
                if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                    ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
            skill['ratingPercentage_5'] = ratings_percentages[0]
            skill['ratingPercentage_4'] = ratings_percentages[1]
            skill['ratingPercentage_3'] = ratings_percentages[2]
            skill['ratingPercentage_2'] = ratings_percentages[3]
            skill['ratingPercentage_1'] = ratings_percentages[4]        
        else:
            print("Rating doesn't exist")
            skill['rating'] = '0'
            skill['ratingPercentage_5'] = '0'
            skill['ratingPercentage_4'] = '0'
            skill['ratingPercentage_3'] = '0'
            skill['ratingPercentage_2'] = '0'
            skill['ratingPercentage_1'] = '0'
        skill['reviewNum'] = review_count_raw[0].get_text().strip()
        skill['description'] = description_raw[0].get_text()
        commands = list()
        for i in range(0,len(commands_raw)):
            commands.append(commands_raw[i].get_text().strip())
        skill['commands'] = commands 
        for i, inv in enumerate(invocation_raw):
            if("Invocation Name" in inv.span.get_text()):
                print("invocation exist")
                skill['invocation'] = invocation_raw[i].span.span.get_text()
                break
            else:
                print('invocation does not exist')
                skill['invocation'] = 'No_Invocation'
        skill['timestamp'] = time.time()
        skills.append(skill)
        cur_skill_Title = skill['title']
        if(soup.select("#dp-summary-see-all-reviews") != []): #review가 있을 때
            reviews_link = 'https://www.amazon.com'+soup.select("#dp-summary-see-all-reviews")[0]['href'] #review link가 없는 경우도 있다 # coin watch처럼
            driver.get(reviews_link)
            driver.find_element_by_css_selector(".reviewer-type-select").click() #파폭(맥북스크린)
            all_reviewers_flag = driver.find_element_by_css_selector("#reviewer-type-dropdown > option").text
            if(all_reviewers_flag == 'All reviewers'):
                print('yes, it is all reviewers')
                driver.find_element_by_css_selector("#reviewer-type-dropdown_0").click()
            else:
                print('wrong filter_not All reviewers')
                break
            while(True):
                time.sleep(2) # wait 2secs
                comments_subComment_toggle_list = driver.find_elements_by_css_selector("div.a-row.a-spacing-top-small.review-comments > div > a") #comments_subComment
                for button in comments_subComment_toggle_list:
                    print(button)
                    button.location_once_scrolled_into_view
                    element = WebDriverWait(driver, 10).until(
                        EC.visibility_of(button)
                    )
                    element.click() #click all subcomments toggle
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                for i, review_data in enumerate(soup.select("#cm_cr-review_list")[0].children):
                    if(review_data['class'] != ['a-section', 'review']): #review가 아니면 탈출, 마지막에 페이지 모듈도 children에 있어서 조건문 담
                        break
                    review = {}
                    review['skill_title'] = cur_skill_Title
                    review['title'] = review_data.div.find_all("a", class_="a-size-base a-link-normal review-title a-color-base a-text-bold")[0].get_text()
                    review['author'] = review_data.div.find_all("a", class_="a-size-base a-link-normal author")[0].get_text() #review_author
                    review['date'] = review_data.div.find_all("span", class_="a-size-base a-color-secondary review-date")[0].get_text() #review date
                    review['rating'] = review_data.div.a['title'] 
                    review['content'] = review_data.div.find_all("span", class_="a-size-base review-text")[0].get_text() #review_content
                    if(review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span is not None): # 추천이 있으면
                        review['vote'] = review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span.get_text().strip() #review_votes, strip()은 앞 뒤 공백제거
                    else:
                        review['vote'] = '0'
                    comments_data = review_data.find_all("div", class_ = "a-section a-spacing-none review-comments")[0]
                    if(comments_data.div is not None): #comment가 있다면
                        print("there are comments")        
                        for i, comment_data in enumerate(comments_data):
                            if(comment_data['class'] == ['a-section', 'a-spacing-small', 'a-spacing-top-small', 'review-comment']):
                                comment = {}   
                                comment['review_title'] = review['title']
                                comment['author'] = comment_data.find(attrs={"data-hook": "review-author"}).get_text() #subcomment author
                                comment['date'] = comment_data.find("span", class_="a-color-tertiary comment-time-stamp").get_text() #subcomment date
                                comment['content'] = comment_data.find("span", class_="review-comment-text").get_text() #subcomment content
                                comments.append(comment)
                    else:
                        print("no comments")
                    review['commentNum'] = review_data.div.find_all("span", class_="review-comment-total aok-hidden")[0].get_text() #comment_num
                    review['timestamp'] = time.time()
                    reviews.append(review)
                if(soup.select("#cm_cr-pagination_bar > ul > li.a-last") != []):
                    if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
                        print('This is not a last page')
                        driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
                    else:
                        print('This is a last page')
                        print('Move to next Skill')
                        break
                else:
                    print('There are no other pages')
                    break
        else: #review가 없을 때
            print('No review link')        
    driver.get(cur_page)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break
skills_df.to_csv('./skills_df.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')
reviews_df.to_csv('./reviews_df.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')
comments_df.to_csv('./comments_df.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

Rating exist
rating_raw = 3.5 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="98dcca27-e87c-40c6-803d-dded1d4ac154")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="bb942147-ecd2-45d2-b7b7-d00fdc7e27a2")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="d73bdcd7-2195-4fe0-8c41-168c607cd045")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="2d0481e6-3922-4036-ab7b-3c7543199e1f")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="88b8c8b9-999c-4876-9fc6-740acda80016")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", elemen

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="f22f5f6d-43b6-47fd-b6cd-d87c910b8460")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="974a5eb5-38f8-4414-9149-7a74e187b77d")>
no comments
no comments
no comments
there are comments
there are comments
there are comments
there are comments
there are comments
there are comments
no comments
This is not a last page
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="a62cf330-f21a-4036-9435-f90acce3a164")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="a36e69a3-60a3-4167-9948-8aaf247db582")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="da471d7e-0187-443f-8e42-0a6266018e15")>
<selenium.webdriver.firefox.webelement.

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="c112e3c5-c45b-4e05-8c8f-8028bd4bf583")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="7e599c98-577d-4632-85ee-c81478ad8ff5")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="16e0cab6-a0af-4f7b-a8c5-611cca3384ee")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="5f56460e-d9b6-4a5c-9dbb-70e8a61c2c31")>
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
This is not a last page
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="1f79c160-ecd7-46d7-8cb0-ef5c2b1ceca5")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="42a5cc48-3ab6-4d4f-8728-39ecd7313b12")>
no comments
no comments
no comments
no comments
no comments
This is a last page
Move to next Skill
Rating exist
rating_raw = 5.0 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="360539c9-afe7-4d55-8a12-897a3acb23dd")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="0db38f03-1ef4-49bc-82e6-419ecef35255")>
no comments
no comments
There are no other pages
Rating exist
rating_raw = 4.0 out of 5 stars
invocation does not exist
invocation does not exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="ec01c5a6-2ac8-4890-ad66-ce8c475b44f2"

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="5304f032-f5a7-4077-a3c0-65655d7e0189")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="d8373234-c00f-4653-b897-2d56df19c751")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="89708ff5-3b7e-4909-a5f8-1fb052c3a00f")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="c3285666-a1d9-49e9-ae5e-d2f009c6f5c1")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="51fce2ab-09b6-437b-8858-3f30aa3f4f12")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="de1d07f3-388a-4f5a-89fd-7425a0d6cf9a")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785

...

yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="7215a4c1-7255-4f57-afb0-75c7df4e6d53")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="ea979930-27a1-4020-8acc-65c107dd53bf")>
no comments
no comments
There are no other pages
Go to next page
Rating exist
rating_raw = 5.0 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="2f857db0-16c1-4478-85c7-526550d1abef")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="0ae79e89-cb06-447d-89a8-acde72fba920")>
no comments
no comments
There are no other pages
Rating doesn't exist
invocation does not exist
invocation exist
No review link
Rating doesn't exist
invocation does not exist
in

Go to next page
Rating exist
rating_raw = 3.8 out of 5 stars
invocation does not exist
invocation does not exist
invocation does not exist
invocation does not exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="cc757462-5033-4049-9015-46e0f2550c66")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="c4e9a868-a525-4000-a90d-6066d45a2f24")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="f97e7a53-3e5b-48ac-aeb5-da0cc8e3bd4a")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="375ba856-1b54-4ddc-8493-14e07781738a")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="a22b6d1d-a517-4682-88d9-c0b854dfd271")>
<selenium.webdriver.firefox.webelem

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="6746aa50-73a6-4a94-8fff-57baf77d04f7")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="9a64f60d-f799-4355-b86c-9ce0ab832554")>
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
There are no other pages
Rating exist
rating_raw = 5.0 out of 5 stars
invocation does not exist
invocation does not exist
invocation does not exist
invocation does not exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="ff6e66a1-05a0-4848-9b77-43786c41ba2e")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="62ff824c-54c4-48b8-abaa-c39b99d2df44")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="f217c045-79ed-4fd8-a57f-851f819d7190")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="1dc52ca7-8303-49a4-9b15-a900fc80e290")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="206cd6c8-a6c5-4dee-a5e4-d2b23e1e9e3e")>
no comments
no comments
no comments
no comments
no comments
no comments
no comments
This is a last page
Move to next Skill
Rating exist
rating_raw = 3.8 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="09792af2-8ad8-4ba9-9e81-8bc5963cccef")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="785dfc05-9cb1-478f-b4fa-00dac02fe0a8", element="c052de2d-3d62-48e0-9ec4-aa351

NoSuchElementException: Message: Unable to locate element: .reviewer-type-select


In [14]:
len(skills)

68

In [15]:
len(reviews)

400

In [19]:
len(comments)

55

In [17]:
skills_df = DataFrame(skills)

In [22]:
skills_df

,author,category,commands,description,invocation,rating,ratingPercentage_1,ratingPercentage_2,ratingPercentage_3,ratingPercentage_4,ratingPercentage_5,reviewNum,timestamp,title
0,"by Capital One Services, LLC",Business & Finance,"[""Alexa, Open Capital One"", ""Alexa, ask Capita...","Ask Capital One about your credit card, checki...",capital one,3.5 out of 5 stars,23%,4%,8%,12%,53%,145,1.503582e+09,Capital One
1,by Andy K.,Business & Finance,"[""Alexa, open bitcoin exchange"", ""Alexa, ask b...",Bitcoin can say the current exchange rate of b...,bitcoin exchange,5.0 out of 5 stars,0%,0%,0%,0%,100%,2,1.503582e+09,Bitcoin Exchange
2,by The Motley Fool,Business & Finance,"[""Alexa, what's my Flash Briefing?"", ""Alexa, w...",Helping the world invest better – one stock at...,No_Invocation,4.0 out of 5 stars,9%,0%,19%,5%,67%,21,1.503583e+09,Motley Fool Stock Watch
3,by wxrdaniel,Business & Finance,"[""Alexa, ask coin base how much is bitcoin?"", ...",Check Realtime bitcoin and ethereum price from...,coin base,4.1 out of 5 stars,0%,0%,33%,17%,50%,6,1.503583e+09,Coinbase Bitcoin/Ethereum Price
4,by Doug Devitre International Inc.,Business & Finance,"[""Alexa Ask Why Remax"", ""Why Join Remax"", ""Why...",This simple Alexa skill was architected to giv...,why remax,5.0 out of 5 stars,0%,0%,0%,0%,100%,4,1.503583e+09,Why RE/MAX
5,by YNAB,Business & Finance,"[""Alexa, ask Why Nab for the balance of Grocer...",Have Alexa help you gain total control of your...,why nab,3.6 out of 5 stars,22%,6%,9%,10%,53%,32,1.503583e+09,YNAB
6,by Sade Smith,Business & Finance,"[""Alexa, ask Coin Watch what is the price of b...",Coin Watch gives the current USD price of the ...,coin watch,0,0,0,0,0,0,0,1.503583e+09,Coin Watch
7,by Doug Devitre International Inc.,Business & Finance,"[""Alexa, open womens council"", ""why should I g...",Getting involved in the Women's Council of REA...,women's council,5.0 out of 5 stars,0%,0%,0%,0%,100%,2,1.503583e+09,Women's Council
8,by IDT,Business & Finance,"[""Alexa, ask BOSS Money What is the fx rate fo...",Use BOSS Revolution Money to get the FX rate f...,boss money,5.0 out of 5 stars,0%,0%,0%,0%,100%,1,1.503583e+09,BOSS Money
9,by Vadim D,Business & Finance,"[""Alexa, what's my Flash Briefing?"", ""Alexa, w...","Fresh out of college, he took his family wine ...",No_Invocation,0,0,0,0,0,0,0,1.503583e+09,Unofficial Gary Vee Flash Briefing


In [20]:
reviews_df = DataFrame(reviews)

In [21]:
comments_df = DataFrame(comments)

In [18]:
skills_df.to_csv('./skills_df_businessFinance_0_68.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [16]:
reviews_df.to_csv('./reviews_df_businessFinance_0_68.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [17]:
comments_df.to_csv('./comments_df_businessFinance_0_68.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')